# Setup

In [1]:
import logging
logging.getLogger("sagemaker.config").setLevel(logging.WARNING)

In [2]:
import sagemaker
sagemaker_session = sagemaker.Session()
region = sagemaker_session.boto_region_name

role = sagemaker.get_execution_role()
bucket_name = 'amdmic-librispeech'
data = f's3://{bucket_name}'

# Train

In [3]:
# TODO: set training params

num_instances = 1
instance_type = 'ml.p2.8xlarge'
train_params = {
    "epochs":1,
    "backend": "gloo",
    # using smaller size for debugging cuda invalid memory access error
    "train-sample-size": 10,
    "test-sample-size": 2,
#     "train-sample-size": 5_000,
#     "test-sample-size": 1_000,
}

In [4]:
from sagemaker.pytorch import PyTorch

estimator = PyTorch(
    entry_point='convtasnet.py',
    source_dir='scripts',
    role=role,
    hyperparameters=train_params,
    instance_count=num_instances,
    instance_type=instance_type,
    py_version="py38",
    framework_version="1.11.0",
)

In [5]:
estimator.fit({"training": data})

INFO:sagemaker.image_uris:image_uri is not presented, retrieving image_uri based on instance_type, framework etc.
INFO:sagemaker:Creating training-job with name: pytorch-training-2024-04-27-22-57-55-952


2024-04-27 22:57:56 Starting - Starting the training job......
2024-04-27 22:58:51 Starting - Preparing the instances for training.........
2024-04-27 23:00:00 Downloading - Downloading input data...
2024-04-27 23:00:45 Downloading - Downloading the training image........................
2024-04-27 23:04:36 Training - Training image download completed. Training in progress..bash: cannot set terminal process group (-1): Inappropriate ioctl for device
bash: no job control in this shell
2024-04-27 19:05:01,129 sagemaker-training-toolkit INFO     Imported framework sagemaker_pytorch_container.training
2024-04-27 19:05:01,196 sagemaker-training-toolkit INFO     No Neurons detected (normal if no neurons installed)
2024-04-27 19:05:01,209 sagemaker_pytorch_container.training INFO     Block until all host DNS lookups succeed.
2024-04-27 19:05:01,211 sagemaker_pytorch_container.training INFO     Invoking user training script.
2024-04-27 19:05:01,477 sagemaker-training-toolkit INFO     Installin

[2024-04-27 19:05:21.458 algo-1:90 INFO utils.py:28] RULE_JOB_STOP_SIGNAL_FILENAME: None
/opt/conda/lib/python3.8/site-packages/smdebug-1.0.26b20230214-py3.8.egg/smdebug/profiler/system_metrics_reader.py:78: SyntaxWarning: "is not" with a literal. Did you mean "!="?
/opt/conda/lib/python3.8/site-packages/smdebug-1.0.26b20230214-py3.8.egg/smdebug/profiler/system_metrics_reader.py:78: SyntaxWarning: "is not" with a literal. Did you mean "!="?
[2024-04-27 19:05:21.956 algo-1:90 INFO profiler_config_parser.py:111] User has disabled profiler.
[2024-04-27 19:05:21.958 algo-1:90 INFO json_config.py:92] Creating hook from json_config at /opt/ml/input/config/debughookconfig.json.
[2024-04-27 19:05:21.958 algo-1:90 INFO hook.py:206] tensorboard_dir has not been set for the hook. SMDebug will not be exporting tensorboard summaries.
[2024-04-27 19:05:21.959 algo-1:90 INFO hook.py:259] Saving to /opt/ml/output/tensors
[2024-04-27 19:05:21.959 algo-1:90 INFO state_store.py:77] The checkpoint config 

UnexpectedStatusException: Error for Training job pytorch-training-2024-04-27-22-57-55-952: Failed. Reason: AlgorithmError: ExecuteUserScriptError:
ExitCode 1
ErrorMessage "RuntimeError: CUDA error: an illegal memory access was encountered
 CUDA kernel errors might be asynchronously reported at some other API call,so the stacktrace below might be incorrect.
 For debugging consider passing CUDA_LAUNCH_BLOCKING=1."
Command "/opt/conda/bin/python3.8 convtasnet.py --backend gloo --epochs 1 --test-sample-size 2 --train-sample-size 10", exit code: 1